In [1]:
import numpy as np
import gym

***
Here I've used version 1 of frozen lake as v0 was out of date. Most things remain the same except some like in this env.reset also gives additional values for debugging purposes. I have decided to keep the slippery mode ON for this excersise

In [88]:
env = gym.make('FrozenLake-v1', render_mode='ansi', desc=None, map_name="4x4", is_slippery=True)

***
Alpha is the learning rate which is used to take the weighted average of the known Q-value and the new learned value. I have decided to keep is a little on the lower side. Gamma is the duscount rate which decides how far into the future do we consider rewards for maximizing the expected return. epsilon is used for exploitaion vs exploration tradeoff. decay rate is less as for the first few episodes I want to explore more. 

In [91]:
alpha = 0.2  
gamma = 0.99  
epsilon = 1.0 
max_epsilon = 1.0 
min_epsilon = 0.01 
decay_rate = 0.005
num_episodes = 20000
max_steps=100
rewards = []

Q = np.zeros([env.observation_space.n, env.action_space.n])

***
Here I've found out another issue with the new version. env.step(action) returns one extra bool value than expected. After some research I found out that it was something related to the possiblity that the episode was truncated due to some condition. Due to lack of complete knowledge about this I've not used truncated value.

In [ ]:
for episode in range(num_episodes):
    state, _ = env.reset()
    step=0
    done = False
    truncated = False
    total_rewards =0

    for step in range(max_steps):
            
        if np.random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q[state, :])

        new_state, reward, done, truncated, info = env.step(action)

        Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[new_state, :]) - Q[state, action])

        total_rewards += reward

        state = new_state
        if( done == True):
            break
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print(Q)


In [ ]:
rewards_per_thousand= np.split(np.array(rewards), num_episodes/1000)
count = 1000
for r in rewards_per_thousand:
    print( count, ":", str(sum(r/1000)))
    count += 1000

***
In running the final test I've only printed the final state

In [ ]:
env.reset()

for episode in range(3):
    state, _ = env.reset()
    step = 0
    done = False
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        action = np.argmax(Q[state,:])
        
        new_state, reward, done, truncated, info = env.step(action)
        
        if done:
            print(env.render())
        state = new_state
env.close()